***4.2.1   Iterators***

In [1]:
primes = [2, 3, 5, 7]
type(primes)

list

In [2]:
iterator = iter(primes)
type(iterator)

list_iterator

In [4]:
next(iterator)

2

In [5]:
next(iterator)

3

In [6]:
next(iterator)
next(iterator)
next(iterator)

StopIteration: 

In [7]:
try:
    next(iterator)
except StopIteration:
    print('No more values')

No more values


***4.2.2   Iterables***

In [9]:
d = {'one': 1,'two': 2,'three': 3}
d

{'one': 1, 'two': 2, 'three': 3}

In [10]:
k = iter(d)
next(k)

'one'

In [11]:
v = iter(d.values())

In [12]:
d.values()

dict_values([1, 2, 3])

In [13]:
next(v)

1

If a dictionary changes in structure because a key is added or removed, then all iterators become invalid and future iterators may exhibit arbitrary changes to the order their contents. On the other hand, changing the value of an existing key does not change the order of the contents or invalidate iterators.

In [14]:
d.pop('three')
next(k)

RuntimeError: dictionary changed size during iteration

***4.2.3   Built-in Iterators***

In [15]:
def double_and_print(x):
    print('xxx', x, '=>', 2*x, 'xxx')
    return 2*x
s = range(3, 7)
doubled = map(double_and_print, s)

In [16]:
next(doubled)

xxx 3 => 6 xxx


6

In [17]:
next(doubled)

xxx 4 => 8 xxx


8

***4.2.4   For Statements***

The for statement in Python operates on iterators. Objects are iterable (an interface) if they have an __iter__ method that returns an iterator. Iterable objects can be the value of the <expression> in the header of a for statement:

for <name> in <expression>:
    <suite>
To execute a for statement, Python evaluates the header <expression>, which must yield an iterable value. Then, the __iter__ method is invoked on that value. Until a StopIteration exception is raised, Python repeatedly invokes the __next__ method on that iterator and binds the result to the <name> in the for statement. Then, it executes the <suite>.

In [20]:
counts = [1, 2, 3]
for i in  counts:
    print(i)

1
2
3


In [21]:
items = counts.__iter__()
try:
    while True:
        item = items.__next__()
        print(item)
except StopIteration:
    pass



1
2
3


***4.2.5   Generators and Yield Statements***

A generator is an iterator returned by a special class of function called a generator function. Generator functions are distinguished from regular functions in that rather than containing return statements in their body, they use yield statement to return elements of a series.

In [40]:
def letters_generator():
    current = 'a'
    while current <= 'd':
        yield current
        current = chr(ord(current) + 1)
a = letters_generator()

In [41]:
a.__next__()

'a'

In [31]:
for letter in a:
    print(letter)

b
c
d


In [38]:
def test():
    i = 0
    while i < 5:
        yield i
        i = 1+i
b = test()

In [48]:
b.__next__()

StopIteration: 

In [49]:
letters = letters_generator()

In [52]:
letters.__next__()
letters.__next__()
letters.__next__()
letters.__next__()

StopIteration: 

In [60]:
class Letters:
    def __init__(self,start='a', end='e'):
        self.start = start
        self.end = end
    def __iter__(self):
        return LetterIter(self.start, self.end)
def LetterIter(m, n):
        s = range(ord(m), ord(n)+1)
        return iter([chr(k) for k in s])
        

In [61]:
b_to_k = Letters('b', 'k')
first_iterator = b_to_k.__iter__()

In [62]:
next(first_iterator)

'b'

In [63]:
next(first_iterator)

'c'

In [64]:
next(first_iterator)

'd'

In [69]:
caps = map(lambda x: x.upper(), b_to_k)

In [70]:
next(caps)

'B'

***4.2.7   Creating Iterables with Yield***

In [71]:
def all_pairs(s):
    for item1 in s:
        for item2 in s:
            yield(item1, item2)
[j for j in all_pairs([1,2,3])]

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]

In [74]:
class LettersWithYield:
    def __init__(self, start='a', end='e'):
        self.start = start
        self.end = end
    def __iter__(self):
        next_letter = self.start
        while next_letter <= self.end:
            yield next_letter
            next_letter = chr(ord(next_letter)+1)
c2f = LettersWithYield('c', 'f')
second_iterator = c2f.__iter__()
[n for n in second_iterator]

['c', 'd', 'e', 'f']

***4.2.8   Iterator Interface***

In [18]:
class LetterIter:
    def __init__(self, start='a', end='e'):
        self.next_letter = start
        self.end = end
    def __next__(self):
        if self.next_letter == self.end:
            raise StopIteration
        letter = self.next_letter
        self.next_letter = chr(ord(letter) +1)
        return letter

In [19]:
letter_iter = LetterIter()
letter_iter.__next__()

'a'

In [20]:
letter_iter.__next__()

'b'

In [21]:
letter_iter.__next__()

'c'

In [22]:
letter_iter.__next__()

'd'

In [23]:
letter_iter.__next__()

StopIteration: 

In [16]:
letter_iter.__next__()

'f'

In [17]:
letter_iter.__next__()

'g'

***4.2.10   Python Streams***

In [3]:
class Stream:
    class empty:
        def __repr__(self):
            return 'Stream.empty'
    empty = empty()
    def __init__(self, first, compute_rest=lambda: empty):
        assert callable(compute_rest), 'compute_rest must be callable'
        self.first = first
        self._compute_rest = compute_rest
    @property
    def rest(self):
        if self._compute_rest is not None:
            self._rest = self._compute_rest()
            self._compute_rest = None
        return self._rest
    def __repr__(self):
        return 'Stream({0}, <...>)'.format(repr(self.first))

In [4]:
s = Stream(1, lambda: Stream(2, lambda: Stream(3)))

In [5]:
s.first

1

In [6]:
s.rest.first

2

In [7]:
def naturals(n):
    return Stream(n, lambda: naturals(n+1))
num = naturals(1)
num.first

1

In [8]:
num.rest

Stream(2, <...>)

In [9]:
def integer_stream(first):
    def compute_rest():
        return integer_stream(first+1)
    return Stream(first, compute_rest)

In [10]:
def map_stream(fn, s):
    if s is Stream.empty:
        return s
    return Stream(fn(s.first), lambda: map_stream(fn, s.rest))
num_2 = map_stream(lambda x: x**2, num)

num_2.rest.rest.first

9

In [11]:
def filter_stream(fn, s):
    if s is Stream.empty:
        return s
    if fn(s.first):
        return Stream(s.first, lambda: filter_stream(fn, s.rest))
    else:
        return filter_stream(fn, s.rest)

In [12]:
num_div_2 = filter_stream(lambda x: x % 2 == 0, num)
num_div_2.rest.first


4

In [13]:
def first_k_as_list(s, k):
    first_k = []
    while k > 0 and s is not Stream.empty:
        first_k.append(s.first)
        s, k = s.rest, k-1
    return first_k
first_k_as_list(num_2, 6)

[1, 4, 9, 16, 25, 36]

In [14]:
def primes(num):
    def not_divible(x):
        return x % num.first != 0
    def compute_rest():
        return primes(filter_stream(not_divible, num.rest))
    return Stream(num.first, compute_rest)
first_k_as_list(primes(naturals(2)), 10)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]